In [3]:
import os, sys
sys.path.insert(0, '/home/michael/projects/strat')
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'strat.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [17]:
from main.models import Coin, Symbol, Candle, SymbolInfo
import pandas as pd
import numpy as np
from django.utils import timezone
from datetime import datetime, timedelta
import const

In [12]:
from connect import connect
from load_data import get_symbols_info, get_coins_info, get_candles, update_klines_missed, update_klines

In [13]:
client = connect()

In [14]:
Candle.objects.all().delete()
update_klines()

In [19]:
fee = 4e-4
def pnl(current_price, diff_price, init_volume):
    return init_volume*(diff_price*(1-fee)-2*current_price*fee)

In [26]:
for symbol in Symbol.objects.all():
    klines = Candle.objects.filter(symbol = symbol)
    data = pd.DataFrame(klines.values('open','high','low','close'))
    data['body'] = np.abs(data['open']-data['close'])
    data['candle'] = data['high']-data['low']
    current_price = (klines.last().open+klines.last().close)/2
    body_mean = data['body'].mean()
    sinfo = SymbolInfo.objects.update_or_create(symbol=symbol, defaults={'pnl_10': pnl(current_price, body_mean, 10)/symbol.max_leverage,
                                                                         'pnl_25': pnl(current_price, body_mean, 25)/symbol.max_leverage,
                                                                         'pnl_50': pnl(current_price, body_mean, 50)/symbol.max_leverage
                                                                        })

In [28]:
SymbolInfo.objects.all().order_by('-pnl_10').values_list('symbol__base__coin', 'pnl_10', 'symbol__max_leverage')

<QuerySet [('YFI', 1.94438588235294, 50), ('BTC', 1.53303009999999, 125), ('MKR', 0.519762583333333, 50), ('DEFI', 0.247965499999999, 50), ('ETH', 0.161518729166667, 100), ('AAVE', 0.0912057411764705, 25), ('BNB', 0.0367547607843138, 75), ('XMR', 0.0356001583333334, 50), ('ZEC', 0.0288385416666666, 50), ('COMP', 0.0272484083333333, 50), ('EGLD', 0.0228941083333334, 50), ('BCH', 0.0218712777777777, 75), ('SOL', 0.0203998333333334, 50), ('KSM', 0.019709825, 50), ('BTCDOM', 0.0193000000000038, 25), ('BNX', 0.0144911529411766, 50), ('TRB', 0.0137167, 25), ('DASH', 0.013711075, 50), ('LTC', 0.0126065111111111, 75), ('FTT', 0.0112047623529412, 25), '...(remaining elements truncated)...']>